In [15]:
import sys
sys.path.append("../src")
import numpy as np
import matplotlib.ticker as ticker
from sklearn.metrics import confusion_matrix, f1_score, matthews_corrcoef, zero_one_loss
import itertools
import seaborn as sns
#sns.color_palette('husl', n_colors=20)
from utils.visual_functions import *
import torch
%matplotlib inline
fig_path="../figure/paper/"
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, FuncFormatter)



cooll_names = ["Drill", "Fan", "Grinder", "Hair", "Hedge", "Lamp", "Sander", "Saw", "Vacuum"]
plaid_names= ['AC', 'CFL', 'Fan', 'Fridge', 'Hairdryer', 'Heater', 'Bulb', 'Laptop', 'Microwave', 'Vacuum', 'Washer']

whited_names= ['CFL','Drill','Fan','FlatIron','GameConsole','HairDryer','Iron','Kettle','LEDLight',
               'LightBulb', 'Massage', 'Microwave', 'Mixer', 'Monitor', 'PowerSupply','ShoeWarmer','Shredder','SolderingIron',
               'Toaster','VacuumCleaner','WaterHeater' ]

## Parameters experiments

### Hyper-parameters

In [16]:
def get_param_results(dataset="cooll", mcc=True):
    model_name='CNN'
    width=50
    
   
    if dataset=="cooll":
        names = cooll_names 
    image_type= "wrg"
    
    results_f1={}
    results_mcc={}
    results_zl={}
    preds = {}
    trues = {}
    currents = {}
    f_max= 0
    fig =figure(columns=1)

  
    eps_params = [5, 1e1, 1e2, 1e3, 1e4, 1e5]
    deltas = [1, 5, 10, 20, 30, width]
    for delta in deltas:
        
        F1=[]
        MCC=[]
        ZL = []
        for eps in eps_params:
            file_name=f"{dataset}_{image_type}_{model_name}_parameters_delta_{str(delta)}_eps_{str(eps)}"
            #print(f" load data for {file_name}")
            f1  = np.load("../results/"+file_name+"_f1.npy")
            zl  = np.load("../results/"+file_name+"_z_one.npy")
            mcc  = np.load("../results/"+file_name+"_mcc.npy")
            if f1>f_max:
                f_max=f1
                ep = eps
                step= delta
            
            F1.append(f1[0]*100)
            MCC.append(mcc[0])
            ZL.append(zl[0])
            
       

        #if delta==10:#plt.plot([1, 10, 20, 30, 40, 50], np.array(MCC) , label = f'$\delta=${delta}', marker='v',markersize=5)
        plt.plot(eps_params, np.array(MCC) , label = f'$\delta=${delta}', marker='v',markersize=5)
        results_mcc[delta]=MCC
        results_f1[delta]=F1
        results_zl[delta]=ZL
    #plt.ylim(60, 100)
    plt.xlabel(' $\lambda$')
    
    #plt.ylabel('Fmacro (\%)')
    
    ax = plt.gca()
    ax.set_xscale('log')
    #plt.axhline(80, linestyle="--",color="red")
    leg = ax.legend()  
   
    #plt.ylabel('No. of misclassfication')
    #savefig(fig_path+f"ZL_{dataset}",format=".pdf")
    
    plt.ylabel('MCC')
    savefig(fig_path+f"MCC_{dataset}",format=".pdf")
    
    print(f" best params for {dataset}: eps={ep}: steps={step}: score={f_max}")
    return results_mcc


In [17]:
results_delta={}
for dataset in ["plaid", "cooll"]:
    results_mcc=get_param_results(dataset=dataset, mcc=False)
    results_delta[dataset]=np.hstack(list(results_mcc.values()))


 best params for plaid: eps=10.0: steps=20: score=[[0.98112746]]
 best params for cooll: eps=1000.0: steps=50: score=[[1.]]


In [18]:
fig =figure(columns=1)
for k, ep in enumerate([5, 1e1, 1e2, 1e3, 1e4]):
    plt.plot([1, 5, 10, 20, 30, 50], results_delta["plaid"][k], label=f'$\epsilon=${1/ep}',  marker='v',markersize=5)
ax = plt.gca()
plt.ylabel('MCC') 
plt.xlabel("$\delta$")
leg = ax.legend()  
savefig(fig_path+f"MCC_plaid_delta",format=".pdf")

In [19]:
fig =figure(columns=1)
for k, ep in enumerate([5, 1e1, 1e2, 1e3, 1e4]):
    #print()
    if k ==0:
        label=f'$\lambda=5$'
    else:
        label=f'$\lambda=10^{int(np.log10(ep))}$'
        
    plt.plot([1, 5, 10, 20, 30, 50], results_delta["cooll"][k], label=label,  marker='v',markersize=5)
ax = plt.gca()
plt.ylabel('MCC') 
plt.xlabel("$\delta$")
leg = ax.legend()  
savefig(fig_path+f"MCC_cooll_delta",format=".pdf")

In [20]:
fig =figure(columns=1)
plt.plot([5, 1e1, 1e2, 1e3, 1e4, 1e5], results_delta["cooll"][:,0], color='C0', label="COOLL-BRP",  marker='v',markersize=5)
plt.plot([5, 1e1, 1e2, 1e3, 1e4, 1e5], results_delta["cooll"][:,-1], color='C1', label="COLL-WRG",  marker='v',markersize=5)
ax = plt.gca()
ax.set_xscale('log')
plt.ylabel('MCC')
#plt.ylim(0.8, 1.001)
#plt.axhline(80, linestyle="--",color="red")
leg = ax.legend()  
plt.xlabel(' $\lambda$')
savefig(fig_path+f"cooll_binary_wrg",format=".pdf")

### Input embending 


In [21]:
def get_param_width_results(dataset="lilac"):
    model_name='CNN'
    width=50
    
   
    if dataset=="cooll":
        names = cooll_names 
    image_type= "wrg"
    
    results_f1={}
    results_mcc={}
    results_zl={}
    preds = {}
    trues = {}
    currents = {}
    results_time = {}
    f_max= 0
    
    widths = [20, 30, 40, 50, 60, 80, 100]
    results = pd.DataFrame( index=widths, columns=["F1", "MCC", "ZL", "TIME"])
    for width in widths:
        file_name=f"{dataset}_{image_type}_{model_name}_parameters_width_{str(width)}"
        
           
        f1  = np.load("../results/"+file_name+"_f1.npy").flatten()
        zl  = np.load("../results/"+file_name+"_z_one.npy").flatten()
        mcc  = np.load("../results/"+file_name+"_mcc.npy").flatten()
        time = np.load("../results/"+file_name+"_time.npy").flatten()
        
        results_f1[width]=list(f1)[0]
        results_mcc[width]=list(mcc)[0]
        results_zl[width]=list(zl)[0]
        results_time[width]=list(time)[0]/60
        
    results["F1"] = pd.Series(results_f1)
    results["MCC"] = pd.Series(results_mcc)
    results["ZL"] = pd.Series(results_zl)
    results["TIME"] = pd.Series(results_time)
    results=results.round(4)
    return results
           
results={}
for dataset in ["plaid", "cooll"]:
    results[dataset]=get_param_width_results(dataset=dataset)

In [22]:
fig =figure(columns=1)
plt.plot(np.array([20, 30, 40, 50, 60, 80, 100]), results['plaid']['MCC'].values, marker='v',markersize=5, label="PLAID ")
plt.plot(np.array([20, 30, 40, 50, 60, 80, 100]), results['cooll']['MCC'].values, marker='v',markersize=5, label="COOLL ")
plt.ylim(0.5, 1.05)
plt.xlabel('$w$')
plt.ylabel('$MCC$')
ax = plt.gca()
leg = ax.legend()
ax.minorticks_on()
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.xaxis.set_minor_locator(MultipleLocator(5))
savefig(fig_path+f"paramaeter_width_plaid_cool", format=".pdf")

In [23]:
fig =figure(columns=1)
plt.plot(np.array([20, 30, 40, 50, 60, 80, 100]), results['plaid']['TIME'].values, marker='v',markersize=5, label="PLAID ")
plt.plot(np.array([20, 30, 40, 50, 60, 80, 100]), results['cooll']['TIME'].values, marker='v',markersize=5, label="COOLL ")
#plt.ylim(70, 100)
plt.xlabel('$w$')
plt.ylabel('Time ($m$)')
ax = plt.gca()
leg = ax.legend()
ax.minorticks_on()
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
ax.xaxis.set_minor_locator(MultipleLocator(5))
savefig(fig_path+f"paramaeter_width_time", format=".pdf")

## General performance

## Generalized performance

In [24]:
def get_model_results(dataset="plaid"):
    model_name='CNN'
    width=50
    cv=4
    
    if dataset =="plaid":
        names = plaid_names
    elif dataset=="cooll":
        names = cooll_names
    
    
    
    results_cm = {}
    results_f1={}
    results_mcc={}
    results_zl={}
    preds = {}
    trues = {}
    currents = {}
    #image_types = ["vi","brgvi", "wrg"]
    image_types = [ "brgvi", "wrg"]
    results = pd.DataFrame( index=image_types, columns=["F1", "MCC", "ZL"])
    for image_type in image_types:
        if dataset=="cooll" and image_type=="brgvi":
            file_name=f"cooll_wrg_CNN_parameters_delta_{str(1)}_eps_{str(10.0)}"
        elif dataset=="plaid" and image_type=="brgvi":
            file_name=f"plaid_wrg_CNN_parameters_delta_{str(1)}_eps_{str(5)}"
        else:
            file_name=f"{dataset}_{image_type}_{model_name}_general_perfomance"
        #file_name=f"{dataset}_{image_type}_{model_name}_objective_one_perfomance"
        

        pred = np.load("../results/"+file_name+"_pred.npy")
        true = np.load("../results/"+file_name+"_true.npy")
        #img = np.load("../results/"+file_name+"_images.npy")
        preds[image_type]=pred
        trues[image_type]=true

        mcc  = matthews_corrcoef(true, pred)
        zl   = zero_one_loss(true, pred)*100
        cm   = confusion_matrix(true, pred)
        _, f1   = get_Fmeasure(cm, names)
        results_cm[image_type]=cm
        results_f1[image_type]=f1
        results_mcc[image_type]=mcc
        results_zl[image_type]=zl
        #f1  = np.load("../results/"+file_name+"_f1.npy")
        #zl  = np.load("../results/"+file_name+"_z_one.npy")
        #mcc  = np.load("../results/"+file_name+"_mcc.npy")
        #print(f'results for {image_type} image type with {dataset} dataset')
        #print(f"mcc:{round(mcc.mean(), 2)}:{round(mcc.std(), 2)}")
        #print(f"f1:{round(f1.mean()*100, 2)}:{round(f1.std()*100, 2)}")
        #print(f"zl:{round(zl.mean(), 2)}:{round(zl.std(), 2)}")
        #print('')
        fig=figure(fig_width=8, fig_height=7)
        plot_confusion_matrix(results_cm[image_type], names, title=None)
        savefig(fig_path+f"cm_{file_name}", format=".pdf")

    fig=figure(fig_width=7,  fig_height=7)
    #plot_multiple_fscore(label_name[dataset], results["vi"][0],results["wrg"][0], labels=["V-I", "WRG"])
    plot_multiple_fscore(names, results_cm["brgvi"],results_cm['wrg'], labels=["RG", "WRG"])
    savefig(fig_path+f"fm_{dataset}_{file_name}", format=".pdf")



    
    results["MCC"] = pd.Series(results_mcc)
    results["F1"] = pd.Series(results_f1)
    results["ZL"] = pd.Series(results_zl)
    results=results.round(2)
    print(f"results for {dataset}")
    print(results)
    print("")
    
    return results

    

In [25]:
for dataset in ["cooll", "plaid"]:
     get_model_results(dataset)

results for cooll
          F1   MCC    ZL
brgvi  98.99  0.98  1.90
wrg    99.86  1.00  0.12

results for plaid
          F1   MCC    ZL
brgvi  88.18  0.91  8.18
wrg    94.35  0.97  2.98



## Generalized performance

In [26]:
label_name = {"whited": whited_names, 
             "cooll":cooll_names,
             'plaid':plaid_names
             }
def generalisation_performance(
                        dataset = "whited",
                        image_type = "wrg",
                            ):
    file_name = f"../results/{dataset}_{image_type}_CNN_generalization_perfomance"
    pred=np.load(f"../results/{file_name}_pred.npy")
    true=np.load(f"../results/{file_name}_true.npy")
    #f1=np.load(f"../results/{file_name}_f1.npy")
    mcc  = matthews_corrcoef(true, pred)
    zl   = zero_one_loss(true, pred)*100
    #f1  = np.load("../results/"+file_name+"_f1.npy")
    #zl  = np.load("../results/"+file_name+"_z_one.npy")
    #mcc  = np.load("../results/"+file_name+"_mcc.npy")
    
    cm = confusion_matrix(true, pred)
    _, f1=get_Fmeasure(cm, label_name[dataset])
    print(f'results for {image_type} image type with {dataset} dataset')
    _, f1=get_Fmeasure(cm, label_name[dataset])
    print(f"f1:{round(f1, 2)}")
    print(f"mcc:{round(mcc, 2)}")
    #print(f"f1:{round(f1.mean()*100, 2)}:{round(f1.std()*100, 2)}")
    print(f"zl:{round(zl, 2)}")
    print('')
    
    fig=figure(fig_width=12) if dataset =="whited" else figure(columns=2)
    plot_confusion_matrix(cm, label_name[dataset], title=None)
    savefig(fig_path+f"cm_{image_type}_{dataset}_generalization", format=".pdf")
    
    #print(f"{dataset}: {image_type} zl: {zl}")
    #print(f"{dataset}: {image_type} mcc: {mcc}")
    #print(f"{dataset}: {image_type} f1: ${round(np.mean(f1), 3)}\pm {round(np.std(f1), 3)}$")
    
    return (cm, mcc, zl)

In [27]:
for dataset in ["cooll", "plaid", "whited"]:
    results = {}
    for image_type in ["vi", "wrg"]:
         results[image_type]= generalisation_performance(dataset, image_type)
            
    fig=figure(fig_width=8, fig_height=7) if dataset == "whited" else figure(fig_width=6, fig_height=5)
    plot_multiple_fscore(label_name[dataset], results["vi"][0],results["wrg"][0], labels=["V-I", "WRG"])
    savefig(fig_path+f"fm_{dataset}_generalization", format=".pdf")

results for vi image type with cooll dataset
f1:98.95
mcc:0.99
zl:1.17

results for wrg image type with cooll dataset
f1:99.86
mcc:1.0
zl:0.17

results for vi image type with plaid dataset
f1:84.75
mcc:0.88
zl:10.71

results for wrg image type with plaid dataset
f1:88.53
mcc:0.92
zl:7.26

results for vi image type with whited dataset
f1:89.63
mcc:0.9
zl:9.29

results for wrg image type with whited dataset
f1:97.23
mcc:0.98
zl:2.29

